In [1]:
wd = "/home/siu/tools/openmmtools/run_protein_ligand_complex"
protein_fn, ligand_fn = "protein.pdb", "ligand.mol" # ligand는 sdf, mol 

In [2]:
import os 
os.chdir(wd)
print(f"YOU ARE HERE: {os.getcwd()}")

YOU ARE HERE: /home/siu/tools/openmmtools/run_protein_ligand_complex


## 파일 불러오기 및 시스템 생성 (토폴로지 생성, 물 분자 추가)

In [3]:
from openmm.app import PDBFile, Modeller, Simulation, DCDReporter, StateDataReporter
from openmm import app, unit, LangevinIntegrator, MonteCarloBarostat
from openmmforcefields.generators import SystemGenerator
from openff.toolkit import Molecule

In [4]:
protein = PDBFile(protein_fn)
ligand = Molecule.from_file(ligand_fn)

In [5]:
# SystemGenerator 생성
system_generator = SystemGenerator(
    forcefields=['amber/ff14SB.xml', 'amber/tip3p_standard.xml'],
    small_molecule_forcefield='gaff-2.11',
    molecules=[ligand],
    forcefield_kwargs={
        'constraints': app.HBonds,
        'rigidWater': True,
        'removeCMMotion': False,
        'hydrogenMass': 4 * unit.amu
    }
)

In [6]:
# Modeller로 단백질+리간드 복합체 생성
modeller = Modeller(protein.topology, protein.positions)   # 모델러 객체 생성 (단백질 좌표, 토폴로지)
ligand_topology = ligand.to_topology()                     # 리간드 토폴로지 생성
modeller.add(ligand_topology.to_openmm(),                  # OpenFF 정보를 OpenMM으로 변환하여 modeller에 추가
             ligand.conformers[0].to_openmm())

with open("system.pdb", "w") as f:
    PDBFile.writeFile(modeller.topology, modeller.positions, f)

In [7]:
# 워터 박스 추가 (PBC 박스와 물 분자를 미리 체크해서 중복 추가 방지) 
from collections import Counter
resnames = [res.name.upper() for res in modeller.topology.residues()] # 1. 현재 topology residue 이름 수집
res_counter = Counter(resnames)
water_resnames = {'HOH', 'WAT', 'TIP3', 'H2O'} # 2. 물 residue 이름 정의
num_waters = sum(res_counter[res] for res in water_resnames) # 3. 물 분자 갯수 확인
box_vectors = modeller.topology.getPeriodicBoxVectors() # 4. 박스 벡터 설정 여부 확인

# 4. 조건 만족 여부 확인
if box_vectors is not None and num_waters >= 100:
    print(f"✅ Water already present ({num_waters} residues) with PBC box. Skipping solvation.")
else:
    print(f"✅ No water found. Adding water with periodic boundary conditions.")
    modeller.addSolvent(
        system_generator.forcefield,
        padding=1.0 * unit.nanometer,
        ionicStrength=0.15 * unit.molar,
        neutralize=True
    )
    print("💧 Water added to the system.")

with open("system_solvated.pdb", "w") as f:
    PDBFile.writeFile(modeller.topology, modeller.positions, f)

✅ No water found. Adding water with periodic boundary conditions.
💧 Water added to the system.


In [8]:
from collections import defaultdict

standard_amino_acids = {
    # 표준 아미노산 (20개)
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
    'GLU', 'GLN', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO',
    'SER', 'THR', 'TRP', 'TYR', 'VAL',

    # N-terminus / C-terminus caps
    'ACE',   # Acetyl cap (N-term)
    'NME',   # N-methylamide cap (C-term)
    'NH2',   # C-terminal amide cap (alternative)
    'FOR',   # Formyl group (N-term cap)

    # 변형 히스티딘 (양성자화 상태 따라 다름)
    'HID', 'HIE', 'HIP',

    # 비표준 아미노산 / 포스트트랜슬레이션 변형
    'SEP',   # Phosphoserine
    'TPO',   # Phosphothreonine
    'PTR',   # Phosphotyrosine
    'MSE',   # Selenomethionine (X-ray 구조에 자주 등장)

    # 기타 생체 관련 유사 잔기
    'CYX',   # Disulfide-bonded cysteine
    'ASX', 'GLX'  # Ambiguous ASP/ASN, GLU/GLN
}

atom_counts = defaultdict(int)
residue_counts = defaultdict(int)

for chain in modeller.topology.chains():
    for residue in chain.residues():
        resname = residue.name.upper()
        if resname in ['HOH', 'WAT', 'TIP3', 'H2O']:  # 물
            category = 'water'
        elif resname in ['NA', 'CL', 'K', 'CA', 'MG']:  # 일반적인 이온
            category = 'ion'
        elif resname in standard_amino_acids:
            category = 'protein'
        else:
            category = 'hetero'  # 리간드, 보조인자 등

        atom_counts[category] += sum(1 for _ in residue.atoms())
        residue_counts[category] += 1

# 출력
for category in ['protein', 'hetero', 'ion', 'water']:
    res_count = residue_counts[category]
    atom_count = atom_counts[category]
    print(f"{category.capitalize():<10}: {res_count} residues ({atom_count} atoms)")

# 전체 합계
total_atoms = sum(atom_counts.values())
total_residues = sum(residue_counts.values())
print(f"\nTotal: {total_residues} residues, {total_atoms} atoms")


Protein   : 110 residues (1705 atoms)
Hetero    : 1 residues (54 atoms)
Ion       : 47 residues (47 atoms)
Water     : 7716 residues (23148 atoms)

Total: 7874 residues, 24954 atoms


## 시스템 최적화 및 평형화

In [9]:
# System 생성
system = system_generator.create_system(modeller.topology, molecules=[ligand])

# Integrator 구성 (Langevin → NVT)
temperature = 300 * unit.kelvin # 자주 쓰이니까 변수로 지정
integrator = LangevinIntegrator(temperature, 1.0 / unit.picosecond, 0.002 * unit.picoseconds)

# Simulation 객체 생성
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
print("Initial system setup complete.")

Initial system setup complete.


In [10]:
%%time
# 에너지 최적화
simulation.minimizeEnergy()

# 최소화 후 구조 저장
with open("system_minimized.pdb", "w") as f:
    PDBFile.writeFile(modeller.topology,
                      simulation.context.getState(getPositions=True).getPositions(),
                      file=f)

print("Energy minimization done. Saved system_minimized.pdb")

Energy minimization done. Saved system_minimized.pdb
CPU times: user 7.91 s, sys: 598 ms, total: 8.5 s
Wall time: 8.49 s


In [11]:
# NVT 평형 - barostat(압력 조절기) 없이 온도만 유지
from tqdm import tqdm
total_steps, chunk_size = 100000, 100
for _ in tqdm(range(0, total_steps, chunk_size), desc="NVT equilibration"):
    simulation.step(chunk_size)

# 📥 시뮬레이션 종료 후 상태 저장 (NVT 결과)
with open("system_nvt.pdb", "w") as f:
    pdb_file = PDBFile.writeFile(simulation.topology,
        simulation.context.getState(getPositions=True).getPositions(),
        file=f
    )

NVT equilibration: 100%|████████████████████████████████| 1000/1000 [00:29<00:00, 33.61it/s]


In [12]:
from openmm import MonteCarloBarostat
# NPT 평형 - barostat를 추가하여 압력을 조절
simulation.context.setVelocitiesToTemperature(temperature) # 속도 재설정 (온도에 맞게 초기화)
simulation.context.getSystem().addForce(
    MonteCarloBarostat(1.0 * unit.atmospheres, temperature, 25)
) # 시스템에 barostat 추가 (1 atm 압력 유지, 25 스텝마다 시도)
for _ in tqdm(range(0, total_steps, chunk_size), desc="NPT equilibration"):
    simulation.step(chunk_size)

# 📥 시뮬레이션 종료 후 상태 저장 (NVT 결과)
with open("system_npt.pdb", "w") as f:
    pdb_file = PDBFile.writeFile(simulation.topology,
        simulation.context.getState(getPositions=True).getPositions(),
        file=f
    )

NPT equilibration: 100%|████████████████████████████████| 1000/1000 [00:29<00:00, 33.45it/s]


## Production Run

In [13]:
# Production run
production_steps = 50000  # 예: 50,000 steps → 100 ps

In [14]:
# 리포터 설정
simulation.reporters.append(DCDReporter("md.dcd", 1000))  # 1000스텝(=2ps)마다 DCD 프레임 저장
simulation.reporters.append(
    StateDataReporter("md.log", 1000,                     # 로그 파일 저장
                      step=True, potentialEnergy=True,
                      temperature=True, density=True, volume=True)
)

In [15]:
for _ in tqdm(range(0, production_steps, chunk_size), desc="Production MD"):
    simulation.step(chunk_size)

print("✅ Production MD complete.")

Production MD: 100%|██████████████████████████████████████| 500/500 [00:15<00:00, 31.76it/s]

✅ Production MD complete.


In [16]:
import mdtraj as md

# 단백질을 중앙에 놓기 (PBC 때문)
traj = md.load("md.dcd", top="system_solvated.pdb")
traj.image_molecules(inplace=True)

# 단백질 구조를 레퍼런스에 정렬시키기
ref = md.load("system_solvated.pdb")
protein = traj.topology.select("protein")
ref_protein = ref.topology.select("protein")
traj.superpose(ref, atom_indices=protein, ref_atom_indices=ref_protein)

# 저장
traj.save_dcd("md_align.dcd")             # 정렬된 trajectory
traj[0].save_pdb("md_align.pdb")    # 정렬된 첫 프레임 (topology용)

In [17]:
print("CHECK YOUR FILES: md_align.pdb and .dcd")

CHECK YOUR FILES: md_align.pdb and .dcd
